In [21]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
import shutil

In [23]:
data_dir = "/media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/"
img_dir = os.path.join(data_dir, "new_label/")
excel_path = os.path.join(os.getcwd(), 'hh_label.xlsx')

df_ex = pd.read_excel(excel_path)

In [24]:
"""
Seperating raw images into 3 categories; surgery, anesthesia, still(useless images) 
"""
# classes directories
# /images
### /surgery
### /anesthesia
###    /handhygiene (target)
### /still

def sort_frames_into_categories(img_dir):
    cnt = 0
    images = glob(img_dir+'*.jpg')
    images.sort()
    
    for image in tqdm(images): 
        img_name= image.split('/')[-1].split('.')[0]
        
        video_id = int(img_name.split('_')[0])
        frame_num = img_name[-6:] # ex: 000001
        
        # classes : surgery, still, target(handhygiene)
        df = df_ex[df_ex['video_id'] == video_id]
        #surgery_list = df['surgery'].iloc[0]
        anesthesia_list = df['anesthesia'].iloc[0]
        still_list = df['still'].iloc[0]
        target_list = df['anesthesia/handhygiene'].iloc[0]
        intubation_list = df['anesthesia/intubation'].iloc[0]
        
        frames = { 'still':still_list, 'anesthesia/handhygiene':target_list, #'surgery': surgery_list,
                 'anesthesia/intubation': intubation_list, 'anesthesia':anesthesia_list}
        
        for procedure in sorted(frames.keys(), reverse=True):
            if frames[procedure] is np.nan:
                continue
            
            lists = frames[procedure].strip().split(',')
            for el in lists:
                start = el.strip().split('-')[0]
                end = el.strip().split('-')[1]
                
                # if current frame is in range of (start, end)
                if int(frame_num) in range(int(start), int(end)+1):
                    if not os.path.exists(image):
                        continue
                        
                    print("image: {}, procedure: {}".format(image, procedure))
                    shutil.move(image, os.path.join(data_dir, 'images/'+procedure))
                    #cnt += 1
                    break
    
    #print("{} images moved to {}".format(cnt, os.path.join(data_dir, 'images/'+procedure)))
    !mv /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/* /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/images/surgery/
    return

In [25]:
sort_frames_into_categories(img_dir)

  5%|▌         | 68/1255 [00:00<00:03, 324.83it/s]

image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001536.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001537.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001538.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001539.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001540.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001541.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001542.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001543.jpg, procedure: anesthesia/handhygiene
imag

 11%|█▏        | 142/1255 [00:00<00:03, 345.83it/s]

image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001607.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames001608.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007443.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007444.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007445.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007446.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007447.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/11_20181126_frames007448.jpg, pr

 17%|█▋        | 214/1255 [00:00<00:02, 349.74it/s]

...

image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094957.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094958.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094959.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094960.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094961.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094962.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_frames094963.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/5_20181119_fram

 96%|█████████▌| 1202/1255 [00:02<00:00, 562.60it/s]

image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001312.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001313.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001314.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001315.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001316.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001317.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames001318.jpg, procedure: anesthesia/handhygiene
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_fram

100%|██████████| 1255/1255 [00:02<00:00, 456.58it/s]


image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018010.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018011.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018012.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018013.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018014.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018015.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018016.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/new_label/8_20181122_frames018017.jpg, procedure: anesthesia
image: /media/minjee/4970a4b3-9bec-42aa-8022-ddf

In [26]:
def images_to_video():
    cnt = 0
    for row in df_ex.values:
        vname = row[1] #video_name
        target = row[6] #anesthesia/handhygiene	
        fps = row[8] #fps
        
        if target is np.nan:
            continue
            
        lists = target.strip().split(',')
        for el in lists:
            start = el.strip().split('-')[0]
            input_dir = os.path.join(data_dir, 'images/anesthesia/handhygiene')
            video_dir = os.path.join(data_dir, "videos/anesthesia/handhygiene")
            images_path = os.path.join(input_dir, vname+'_frames%06d.jpg')
            target_dir = os.path.join(video_dir, vname+'_'+str(cnt)+'.mp4')
            !ffmpeg -r $fps -start_number $start -i $images_path -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $target_dir -y
            cnt += 1

In [27]:
images_to_video()

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

frame=   55 fps=0.0 q=-1.0 Lsize=     585kB time=00:00:03.53 bitrate=1356.3kbits/s    
video:584kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.250360%
[libx264 @ 0x1c12c40] frame I:1     Avg QP:19.65  size: 50907
[libx264 @ 0x1c12c40] frame P:26    Avg QP:20.52  size: 16625
[libx264 @ 0x1c12c40] frame B:28    Avg QP:23.48  size:  4060
[libx264 @ 0x1c12c40] consecutive B-frames:  1.8% 87.3% 10.9%  0.0%
[libx264 @ 0x1c12c40] mb I  I16..4:  4.9% 48.2% 46.8%
[libx264 @ 0x1c12c40] mb P  I16..4:  0.7%  7.6%  3.5%  P16..4: 35.1% 21.7% 18.3%  0.0%  0.0%    skip:13.2%
[libx264 @ 0x1c12c40] mb B  I16..4:  0.1%  0.8%  0.3%  B16..8: 29.2% 11.2%  4.6%  direct: 4.3%  skip:49.5%  L0:36.6% L1:48.5% BI:14.9%
[libx264 @ 0x1c12c40] 8x8 transform intra:61.0% inter:61.8%
[libx264 @ 0x1c12c40] coded y,uvDC,uvAC intra: 81.5% 92.8% 74.2% inter: 29.2% 34.5% 7.5%
[libx264 @ 0x1c12c40] i16 v,h,dc,p: 15% 19%  6% 61%
[libx264 @ 0x1c12c40] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 21% 16% 14

frame=   63 fps=0.0 q=-1.0 Lsize=     453kB time=00:00:04.06 bitrate= 912.6kbits/s    
video:451kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.344360%
[libx264 @ 0x22aeac0] frame I:1     Avg QP:18.52  size: 54775
[libx264 @ 0x22aeac0] frame P:20    Avg QP:19.15  size: 15757
[libx264 @ 0x22aeac0] frame B:42    Avg QP:22.15  size:  2183
[libx264 @ 0x22aeac0] consecutive B-frames:  1.6%  9.5% 57.1% 31.7%
[libx264 @ 0x22aeac0] mb I  I16..4:  5.0% 50.4% 44.6%
[libx264 @ 0x22aeac0] mb P  I16..4:  0.9%  3.9%  2.3%  P16..4: 43.1% 21.3% 19.1%  0.0%  0.0%    skip: 9.3%
[libx264 @ 0x22aeac0] mb B  I16..4:  0.0%  0.2%  0.2%  B16..8: 31.1%  5.1%  1.9%  direct: 2.7%  skip:58.8%  L0:43.2% L1:48.3% BI: 8.5%
[libx264 @ 0x22aeac0] 8x8 transform intra:53.3% inter:71.6%
[libx264 @ 0x22aeac0] coded y,uvDC,uvAC intra: 79.1% 86.4% 64.7% inter: 20.2% 27.6% 5.4%
[libx264 @ 0x22aeac0] i16 v,h,dc,p: 14% 31%  7% 48%
[libx264 @ 0x22aeac0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 24% 18% 16

frame=   16 fps=0.0 q=-1.0 Lsize=     128kB time=00:00:00.93 bitrate=1123.7kbits/s    
video:127kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.790344%
[libx264 @ 0x94b1a0] frame I:1     Avg QP:19.05  size: 48953
[libx264 @ 0x94b1a0] frame P:5     Avg QP:20.59  size: 11337
[libx264 @ 0x94b1a0] frame B:10    Avg QP:22.61  size:  2374
[libx264 @ 0x94b1a0] consecutive B-frames:  6.2%  0.0% 93.8%  0.0%
[libx264 @ 0x94b1a0] mb I  I16..4:  7.0% 51.4% 41.6%
[libx264 @ 0x94b1a0] mb P  I16..4:  0.6%  5.2%  3.3%  P16..4: 38.4% 14.9% 12.8%  0.0%  0.0%    skip:24.9%
[libx264 @ 0x94b1a0] mb B  I16..4:  0.0%  0.4%  0.4%  B16..8: 25.9%  5.8%  1.8%  direct: 2.5%  skip:63.2%  L0:33.9% L1:58.4% BI: 7.7%
[libx264 @ 0x94b1a0] 8x8 transform intra:53.2% inter:68.5%
[libx264 @ 0x94b1a0] coded y,uvDC,uvAC intra: 84.4% 90.6% 74.0% inter: 15.6% 21.3% 3.9%
[libx264 @ 0x94b1a0] i16 v,h,dc,p: 15% 30%  9% 47%
[libx264 @ 0x94b1a0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 22% 23% 14%  4%  6%  

frame=   24 fps=0.0 q=-1.0 Lsize=     213kB time=00:00:01.46 bitrate=1186.9kbits/s    
video:211kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.519214%
[libx264 @ 0x1bba200] frame I:1     Avg QP:19.50  size: 47890
[libx264 @ 0x1bba200] frame P:10    Avg QP:20.65  size: 12959
[libx264 @ 0x1bba200] frame B:13    Avg QP:23.43  size:  2947
[libx264 @ 0x1bba200] consecutive B-frames:  8.3% 41.7% 50.0%  0.0%
[libx264 @ 0x1bba200] mb I  I16..4:  3.3% 55.1% 41.6%
[libx264 @ 0x1bba200] mb P  I16..4:  0.5%  4.1%  2.8%  P16..4: 37.4% 22.1% 16.1%  0.0%  0.0%    skip:17.0%
[libx264 @ 0x1bba200] mb B  I16..4:  0.0%  0.2%  0.1%  B16..8: 30.1%  8.6%  3.3%  direct: 3.4%  skip:54.2%  L0:30.8% L1:55.5% BI:13.7%
[libx264 @ 0x1bba200] 8x8 transform intra:55.1% inter:61.9%
[libx264 @ 0x1bba200] coded y,uvDC,uvAC intra: 85.5% 92.8% 76.2% inter: 22.3% 29.0% 6.3%
[libx264 @ 0x1bba200] i16 v,h,dc,p: 26% 10% 11% 52%
[libx264 @ 0x1bba200] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 24% 19% 12

frame=   23 fps=0.0 q=-1.0 Lsize=     186kB time=00:00:01.40 bitrate=1090.3kbits/s    
video:185kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.586229%
[libx264 @ 0xe31240] frame I:1     Avg QP:19.12  size: 54385
[libx264 @ 0xe31240] frame P:9     Avg QP:20.83  size: 11338
[libx264 @ 0xe31240] frame B:13    Avg QP:22.84  size:  2505
[libx264 @ 0xe31240] consecutive B-frames:  8.7% 26.1% 65.2%  0.0%
[libx264 @ 0xe31240] mb I  I16..4:  5.4% 50.2% 44.4%
[libx264 @ 0xe31240] mb P  I16..4:  0.8%  4.8%  2.8%  P16..4: 42.1% 15.8% 14.2%  0.0%  0.0%    skip:19.5%
[libx264 @ 0xe31240] mb B  I16..4:  0.1%  0.5%  0.3%  B16..8: 27.1%  5.8%  2.3%  direct: 2.6%  skip:61.3%  L0:36.5% L1:54.3% BI: 9.2%
[libx264 @ 0xe31240] 8x8 transform intra:53.6% inter:70.9%
[libx264 @ 0xe31240] coded y,uvDC,uvAC intra: 82.7% 89.2% 71.7% inter: 20.3% 25.2% 4.1%
[libx264 @ 0xe31240] i16 v,h,dc,p:  4% 27%  8% 62%
[libx264 @ 0xe31240] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 18% 25% 12%  4%  7%  

frame=   24 fps=0.0 q=-1.0 Lsize=     176kB time=00:00:01.46 bitrate= 985.5kbits/s    
video:175kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.626006%
[libx264 @ 0xafc200] frame I:1     Avg QP:18.81  size: 54701
[libx264 @ 0xafc200] frame P:8     Avg QP:20.31  size: 11732
[libx264 @ 0xafc200] frame B:15    Avg QP:22.39  size:  2020
[libx264 @ 0xafc200] consecutive B-frames:  8.3%  0.0% 75.0% 16.7%
[libx264 @ 0xafc200] mb I  I16..4:  3.6% 52.5% 43.9%
[libx264 @ 0xafc200] mb P  I16..4:  0.6%  3.1%  2.1%  P16..4: 42.0% 16.3% 14.0%  0.0%  0.0%    skip:21.9%
[libx264 @ 0xafc200] mb B  I16..4:  0.0%  0.2%  0.1%  B16..8: 27.0%  5.3%  1.9%  direct: 2.1%  skip:63.4%  L0:42.9% L1:49.0% BI: 8.1%
[libx264 @ 0xafc200] 8x8 transform intra:52.9% inter:71.1%
[libx264 @ 0xafc200] coded y,uvDC,uvAC intra: 87.1% 93.2% 74.3% inter: 17.5% 23.1% 4.0%
[libx264 @ 0xafc200] i16 v,h,dc,p:  5% 30%  5% 61%
[libx264 @ 0xafc200] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 20% 26% 11%  6%  5%  

frame=   24 fps=0.0 q=-1.0 Lsize=     154kB time=00:00:01.46 bitrate= 858.0kbits/s    
video:153kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.719715%
[libx264 @ 0x1098200] frame I:1     Avg QP:18.31  size: 51843
[libx264 @ 0x1098200] frame P:6     Avg QP:19.86  size: 11452
[libx264 @ 0x1098200] frame B:17    Avg QP:22.28  size:  2055
[libx264 @ 0x1098200] consecutive B-frames:  4.2%  0.0% 12.5% 83.3%
[libx264 @ 0x1098200] mb I  I16..4:  4.6% 52.7% 42.8%
[libx264 @ 0x1098200] mb P  I16..4:  0.2%  2.9%  2.2%  P16..4: 39.4% 16.4% 16.1%  0.0%  0.0%    skip:22.8%
[libx264 @ 0x1098200] mb B  I16..4:  0.0%  0.1%  0.1%  B16..8: 28.4%  5.0%  1.7%  direct: 1.8%  skip:62.8%  L0:39.2% L1:52.0% BI: 8.8%
[libx264 @ 0x1098200] 8x8 transform intra:53.3% inter:71.3%
[libx264 @ 0x1098200] coded y,uvDC,uvAC intra: 88.4% 90.8% 73.0% inter: 14.5% 19.9% 3.1%
[libx264 @ 0x1098200] i16 v,h,dc,p: 18% 31%  4% 47%
[libx264 @ 0x1098200] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 19% 24% 14

[image2 @ 0xa4b5a0] Could not open file : /media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/images/anesthesia/handhygiene/17_20181203_frames064006.jpg
/media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/images/anesthesia/handhygiene/17_20181203_frames%06d.jpg: Input/output error
frame=   22 fps=0.0 q=-1.0 Lsize=     160kB time=00:00:01.33 bitrate= 983.4kbits/s    
video:159kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.675659%
[libx264 @ 0xa59700] frame I:1     Avg QP:18.90  size: 53012
[libx264 @ 0xa59700] frame P:7     Avg QP:20.19  size: 11336
[libx264 @ 0xa59700] frame B:14    Avg QP:22.62  size:  2125
[libx264 @ 0xa59700] consecutive B-frames:  9.1%  0.0% 54.5% 36.4%
[libx264 @ 0xa59700] mb I  I16..4:  6.0% 52.7% 41.3%
[libx264 @ 0xa59700] mb P  I16..4:  0.2%  3.7%  2.0%  P16..4: 40.5% 15.5% 13.3%  0.0%  0.0%    skip:24.9%
[libx264 @ 0xa59700] mb B  I16..4:  0.0%  0.1%  0.2%  B16..8: 26.9%  5.2%  1.9%  direct: 2.1%  skip:63.6%  L0:34.3% L1:58.1% 

frame=   20 fps=0.0 q=-1.0 Lsize=     194kB time=00:00:01.20 bitrate=1324.1kbits/s    
video:193kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.544694%
[libx264 @ 0x25a0220] frame I:1     Avg QP:19.64  size: 55440
[libx264 @ 0x25a0220] frame P:9     Avg QP:21.08  size: 11974
[libx264 @ 0x25a0220] frame B:10    Avg QP:23.28  size:  3365
[libx264 @ 0x25a0220] consecutive B-frames: 10.0% 60.0% 30.0%  0.0%
[libx264 @ 0x25a0220] mb I  I16..4:  2.6% 45.6% 51.8%
[libx264 @ 0x25a0220] mb P  I16..4:  0.7%  7.4%  3.2%  P16..4: 33.1% 17.2% 14.4%  0.0%  0.0%    skip:23.9%
[libx264 @ 0x25a0220] mb B  I16..4:  0.1%  0.7%  0.3%  B16..8: 25.2%  8.4%  3.6%  direct: 3.5%  skip:58.0%  L0:28.1% L1:59.5% BI:12.3%
[libx264 @ 0x25a0220] 8x8 transform intra:55.7% inter:63.1%
[libx264 @ 0x25a0220] coded y,uvDC,uvAC intra: 82.3% 92.4% 75.4% inter: 22.2% 26.5% 6.0%
[libx264 @ 0x25a0220] i16 v,h,dc,p: 36% 17%  6% 40%
[libx264 @ 0x25a0220] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 27% 17% 14